In [1]:
import numpy as np
from scipy.optimize import linprog

In [2]:
margins = np.array([2.15, 1.34, 1.72])
c = -margins
A = np.array([
    [7/10, 1/3, 1/2],
    [1/5, 2/3, 1/6],
    [1/10, 0.0, 1/3]
])
b = np.array([8000, 3000, 2500])

In [3]:
sol = linprog(c, A_ub=A, b_ub=b)
print(sol.x)

[7105.26315789 1026.31578947 5368.42105263]


In [10]:
import pyomo.environ as pyomo
model = pyomo.ConcreteModel()
model.I = pyomo.Set(initialize=['A', 'B', 'C'])
model.J = pyomo.Set(initialize=['D', 'E', 'F'])

model.b = pyomo.Param(model.I, initialize=dict(zip(model.I, b)))
model.c = pyomo.Param(model.J, initialize=dict(zip(model.J, margins)))

proportions = {}
for k, i in enumerate(model.I):
    for l, j in enumerate(model.J):
        proportions[i, j] = A[k, l]

model.f = pyomo.Param(model.I, model.J, initialize=proportions)
model.x = pyomo.Var(model.J, within=pyomo.NonNegativeReals, name='x')

# Availability constraint
def availability_rule_cstr(model, i):
    return sum(model.f[i, j] * model.x[j] for j in model.J) <= model.b[i]

model.cstr_available = pyomo.Constraint(model.I, rule=availability_rule_cstr, name="cstr_available")

# Objective function
def obj_func(model):
    return sum(model.x[j] * model.c[j] for j in model.J)

model.obj = pyomo.Objective(rule=obj_func, sense=pyomo.maximize)

cbc = pyomo.SolverFactory("cbc")
sol = cbc.solve(model, tee=False)

print(model.x.display())
print(model.obj.display())

x : Size=3, Index=J
    Key : Lower : Value     : Upper : Fixed : Stale : Domain
      D :     0 : 7105.2632 :  None : False : False : NonNegativeReals
      E :     0 : 1026.3158 :  None : False : False : NonNegativeReals
      F :     0 : 5368.4211 :  None : False : False : NonNegativeReals
None
obj : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 25885.263344
None


In [4]:
import pyomo.environ as pyomo

In [8]:
model = pyomo.ConcreteModel()
model.x1 = pyomo.Var(domain=pyomo.NonNegativeReals)
model.x2 = pyomo.Var(domain=pyomo.NonNegativeReals)

model.c = pyomo.ConstraintList()
model.c.add(model.x1*10 + 1 >= model.x2)
model.c.add(model.x1*0.2 + 4 >= model.x2)
model.c.add(model.x1 * -0.2 + 6 >= model.x2)

model.objective = pyomo.Objective(rule=lambda model: model.x1 + model.x2*10, sense=pyomo.maximize)

solver = pyomo.SolverFactory('cbc')
result = solver.solve(model)
print(result)
print(model.x1())
print(model.x2())


Problem: 
- Name: unknown
  Lower bound: 55.0
  Upper bound: 55.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Number of nonzeros: 0
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.022161006927490234
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

5.0
5.0
